In [8]:
import os
import time
import requests

ELEFAN_PRODUCT_API_URL = os.environ.get("API_URL")
OFF_API_URL = "https://fr.openbeautyfacts.org/api/v2/product"
OFF_KEY = "OBF"

ELEFAN_HYGIENE_FAMILLE_CODE_TO_INCLUDE = [
    21,  # bebe (inclus nourriture...)
    22,  # hygiène, beauté
    25,  # santé
]

In [9]:
"""
Récupérer les produits depuis Elefan (via l'API)
"""
print("Elefan Grenoble")
elefan_product_list = list()

response = requests.get(ELEFAN_PRODUCT_API_URL)
elefan_product_list = response.json()
print("Nombre d'articles (dans KASO) :", len(elefan_product_list))
# print(elefan_product_list[0])

elefan_product_actifs_list = [p for p in elefan_product_list if p['status'] == 'ACTIF']
print("Nombre d'articles actifs (status = ACTIF dans KASO) :", len(elefan_product_actifs_list))

elefan_product_actifs_ean_list = [p for p in elefan_product_actifs_list if (len(str(p['code'])) in [8, 13])]
print("Nombre d'articles actifs avec un code-barres EAN (8 ou 13 chiffres) :", len(elefan_product_actifs_ean_list))

elefan_product_actifs_ean_filtered_list = [p for p in elefan_product_actifs_ean_list if p['famille']['code'] in ELEFAN_HYGIENE_FAMILLE_CODE_TO_INCLUDE]
print("Nombre d'articles actifs avec un code-barres EAN et filtrés (catégories) :", len(elefan_product_actifs_ean_filtered_list))

Elefan Grenoble
Nombre d'articles (dans KASO) : 2106
Nombre d'articles actifs (status = ACTIF dans KASO) : 1456
Nombre d'articles actifs avec un code-barres EAN (8 ou 13 chiffres) : 1124
Nombre d'articles actifs avec un code-barres EAN et filtrés (catégories) : 137


In [10]:
print(f"Lien avec {OFF_KEY} (pour les {len(elefan_product_actifs_ean_filtered_list)} articles filtrés)")

found = 0
for index, elefan_product in enumerate(elefan_product_actifs_ean_filtered_list):
    response = requests.get(f"{OFF_API_URL}/{elefan_product['code']}.json")
    if response.status_code == 200:
        found += 1
        elefan_product_actifs_ean_filtered_list[index][OFF_KEY] = response.json()
    if (index % 10) == 0:
        time.sleep(1)
    if (index % 50) == 0:
        print(f"progress: {index}...")

print("Trouvés", found, "/", len(elefan_product_actifs_ean_filtered_list))

Lien avec OBF (pour les 137 articles filtrés)
progress: 0...
progress: 50...
progress: 100...
Trouvés 26 / 137


In [12]:
elefan_product_actifs_ean_filtered_found_list = [p for p in elefan_product_actifs_ean_filtered_list if OFF_KEY in p]
print(f"Sur les {len(elefan_product_actifs_ean_filtered_found_list)} produits trouvés dans {OFF_KEY}...")

for product in elefan_product_actifs_ean_filtered_found_list:
    print({key: product[key] for key in ["code", "designation"]})

Sur les 26 produits trouvés dans OFF...
{'code': 3517360005509, 'designation': 'DEODORANT FRAICHEUR MENTHE'}
{'code': 3517360005516, 'designation': 'DEODORANT DOUCEUR LAIT D ANESSE'}
{'code': 3517360011418, 'designation': 'SAVON OLIVE NIGELLE'}
{'code': 3517360011449, 'designation': 'GEL DOUCHE DOUCEUR AMANDE - PECHE'}
{'code': 3517360013757, 'designation': 'GEL HYDRATATION ALOE VERA'}
{'code': 3517360016994, 'designation': 'GEL LAVANT TRES DOUX'}
{'code': 3517360017076, 'designation': 'DEODORANT PEAUX SENSIBLES ALOE VERA'}
{'code': 3662217000630, 'designation': 'FOND DE TEINT NUDE'}
{'code': 3662217000784, 'designation': 'CRAYON YEUX NOIR'}
{'code': 3662217001729, 'designation': 'CREME DE JOUR PEAUX NORMALES A MIXTES'}
{'code': 3662217001774, 'designation': 'LAIT CORPOREL'}
{'code': 3662217002283, 'designation': 'LOTION MICELLAIRE'}
{'code': 3662217003891, 'designation': 'BAUME A LEVRE'}
{'code': 3662217003983, 'designation': 'CRAYON YEUX BRONZE CUIVRE'}
{'code': 3662217004805, 'desig